# CLIP4CAD-GFA Training Notebook

This notebook provides a persistent training environment that keeps the 204GB dataset in memory between runs.

## Key Benefits
- **Load data ONCE** (~5 min) and keep it in RAM
- **Fast iteration** - Restart training instantly
- **No memory overflow** - Using `num_workers=0` since data is in RAM

## Usage
1. Run Cells 1-5 once to load data and initialize model (~5 min)
2. Run Cell 6 to start/restart training (instant!)
3. Use Cell 7-9 for resuming, validation, or monitoring

## Cell 1: Imports & Setup

In [1]:
import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, 'D:/Defect_Det/MMCAD/MMCAD')

import torch
from omegaconf import OmegaConf
from clip4cad.models.clip4cad_gfa import CLIP4CAD_GFA
from clip4cad.data.gfa_dataset import GFAMappedDataset
from clip4cad.training.gfa_trainer import GFATrainer

print("✓ Imports loaded successfully")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

✓ Imports loaded successfully
✓ PyTorch version: 2.5.1+cu121
✓ CUDA available: True


## Cell 2: Configuration

Configure paths, hyperparameters, and output directory.

**CRITICAL**: `num_workers=0` because all data is loaded into RAM (no need for worker processes)

In [ ]:
# Paths
data_root = Path("d:/Defect_Det/MMCAD/data")
pc_file = "c:/Users/User/Desktop/pc_embeddings_full.h5"
brep_file = "c:/Users/User/Desktop/brep_features.h5"
text_file = "c:/Users/User/Desktop/text_embeddings.h5"
output_dir = Path("outputs/gfa_111k")
output_dir.mkdir(parents=True, exist_ok=True)

# Load config
config = OmegaConf.load("d:/Defect_Det/MMCAD/MMCAD/configs/model/clip4cad_gfa.yaml")

# Override training settings
config.training.batch_size = 512
config.training.num_workers = 0  # CRITICAL: No workers since data in RAM!
config.training.num_epochs_stage1 = 15
config.training.num_epochs_stage2 = 20

# Save config
OmegaConf.save(config, output_dir / "config.yaml")

print("✓ Configuration loaded")
print(f"  - Batch size: {config.training.batch_size}")
print(f"  - Num workers: {config.training.num_workers}")
print(f"  - Stage 1 epochs: {config.training.num_epochs_stage1}")
print(f"  - Stage 2 epochs: {config.training.num_epochs_stage2}")
print(f"  - Output dir: {output_dir}")

✓ Configuration loaded
  - Batch size: 512
  - Num workers: 0
  - Stage 1 epochs: 15
  - Stage 2 epochs: 20
  - Output dir: outputs\gfa_111k


## Cell 3: Load Datasets (RUN ONCE, ~5 min)

**This cell loads 204GB of data into RAM**  
**⚠️ Takes ~5 minutes, but only run ONCE!**

After running this cell:
- Training data stays in memory
- Can restart training instantly by re-running Cell 6
- Don't re-run this cell unless you restart the kernel

In [4]:
print("Loading datasets (this takes ~5 min, but only once!)...")
print("="*60)

# Train dataset - LOAD TO RAM
print("\n[1/2] Loading TRAIN dataset to RAM...")
train_dataset = GFAMappedDataset(
    data_root=str(data_root),
    split="train",
    pc_file=pc_file,
    text_file=text_file,
    brep_file=brep_file,
    num_rotations=1,
    load_to_memory=True,  # Load 111k samples to RAM
    use_live_text=False,
)
print(f"✓ Train dataset loaded: {len(train_dataset):,} samples in RAM")

# Val dataset - KEEP ON DISK (saves RAM)
print("\n[2/2] Loading VAL dataset (on disk)...")
val_dataset = GFAMappedDataset(
    data_root=str(data_root),
    split="val",
    pc_file=pc_file,
    text_file=text_file,
    brep_file=brep_file,
    num_rotations=1,
    load_to_memory=False,  # Val stays on disk (saves RAM)
    use_live_text=False,
)
print(f"✓ Val dataset loaded: {len(val_dataset):,} samples on disk")

print("\n" + "="*60)
print("✓ DATASETS READY!")
print(f"  - Train: {len(train_dataset):,} samples in RAM")
print(f"  - Val: {len(val_dataset):,} samples on disk")
print("\n⚠️  Do NOT re-run this cell unless you restart the kernel!")
print("="*60)

Loading datasets (this takes ~5 min, but only once!)...

[1/2] Loading TRAIN dataset to RAM...
  Loading train data to memory (B-Rep + PC + Text)...
    Loading B-Rep (3GB)...
    Loading PC (50GB)...
    Loading Text from: c:\Users\User\Desktop\text_splits\train_text_embeddings.h5
    ✓ Text loaded: 174.6GB in 361.4s
    ⚠️  Pre-split has 111000 samples, dataset expected 133105
    Using first 111000 samples to match pre-split file
  ✓ Loaded 111000 samples: 203.6GB in RAM (B-Rep + PC + Text)
GFAMappedDataset: train with 111000 samples (in memory)
✓ Train dataset loaded: 111,000 samples in RAM

[2/2] Loading VAL dataset (on disk)...
GFAMappedDataset: val with 16638 samples
✓ Val dataset loaded: 16,638 samples on disk

✓ DATASETS READY!
  - Train: 111,000 samples in RAM
  - Val: 16,638 samples on disk

⚠️  Do NOT re-run this cell unless you restart the kernel!


## Cell 4: Initialize Model

Create the CLIP4CAD-GFA model and move it to GPU.

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create model
model = CLIP4CAD_GFA(config)

print("\n✓ Model initialized")
print(f"  - Total parameters: {model.count_parameters():,}")
print(f"  - Trainable parameters: {model.count_parameters(trainable_only=True):,}")

Using device: cuda

✓ Model initialized
  - Total parameters: 3,168,771
  - Trainable parameters: 3,168,771


## Cell 5: Create Trainer

Initialize the GFA trainer with datasets, model, and configuration.

In [38]:
trainer = GFATrainer(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    config=config.training,
    device=device,
    output_dir=str(output_dir),
)

print("✓ Trainer initialized")
print(f"  - Output directory: {output_dir}")
print(f"  - Device: {device}")
print("\n✓ Ready to train! Run Cell 6 to start training.")

✓ Trainer initialized
  - Output directory: outputs\gfa_111k
  - Device: cuda

✓ Ready to train! Run Cell 6 to start training.


In [39]:
# Cell 5.5: Load from Best Checkpoint                                                                                                                                                                                                        checkpoint_path = Path("notebooks/outputs/gfa_111k/checkpoint_best.pt")
checkpoint_path = Path("d:/Defect_Det/MMCAD/MMCAD/notebooks/outputs/gfa_111k/checkpoint_best.pt")
if checkpoint_path.exists():
    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Load model state
    model.load_state_dict(checkpoint['model_state_dict'])
    trainer.resume(str(checkpoint_path))
    print(f"✓ Will continue from epoch {trainer.current_epoch + 1}")
    
else:
    print(f"No checkpoint found at {checkpoint_path}")

Loading checkpoint: d:\Defect_Det\MMCAD\MMCAD\notebooks\outputs\gfa_111k\checkpoint_best.pt


C:\Users\User\AppData\Local\Temp\ipykernel_9736\790574118.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Resuming from: d:\Defect_Det\MMCAD\MMCAD\notebooks\outputs\gfa_111k\checkpoint_best.pt


D:\Defect_Det/MMCAD/MMCAD\clip4cad\training\gfa_trainer.py:520: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=self.devi

Resumed at epoch 15, stage 1
✓ Will continue from epoch 16


## Cell 6: Train (Re-run this cell to restart training)

**Start or restart training from scratch**

This cell can be run multiple times:
- Data stays in RAM (no reload needed!)
- Model re-initializes from scratch
- To resume from checkpoint, use Cell 7 instead

In [40]:
# Optional: Resume from checkpoint
# Uncomment the lines below to resume from a specific checkpoint
# checkpoint_path = output_dir / "checkpoint_epoch5.pt"
# if checkpoint_path.exists():
#     trainer.load_checkpoint(str(checkpoint_path))
#     print(f"✓ Resumed from {checkpoint_path}")

print("Starting training...")
print("="*60)
trainer.train()
print("="*60)
print("✓ Training completed!")

Starting training...
CLIP4CAD-GFA Training
Total epochs: 35
Stage 1: 15 epochs (grounding)
Stage 2: 20 epochs (alignment)
Batch size: 512
Learning rate: 3e-05
Checkpoint every: 5 epochs
Trainable parameters: 3,168,771
Resuming from epoch 16


✓ Saved Stage 1 final checkpoint: outputs\gfa_111k\checkpoint_stage1_final.pt

Transitioning to Stage 2

Mining Hard Negatives


Extracting embeddings:   0%|          | 0/216 [00:00<?, ?it/s]D:\Defect_Det/MMCAD/MMCAD\clip4cad\training\hard_negative_mining.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad():
Extracting embeddings: 100%|██████████| 216/216 [00:56<00:00,  3.85it/s]


Mining hard negatives for 110592 samples...
  Computing geometric kNN...
  Filtering by text similarity...


  Filtering: 100%|██████████| 110592/110592 [00:04<00:00, 25521.34it/s]


  Found hard negatives for 105570 samples
  Average negatives per sample: 6.8
Saved hard negatives to outputs\gfa_111k\hard_negatives\hard_negatives_epoch15.json

Learning rate reduced to: 0.00e+00



...


  [Conf] min=0.007 max=0.993 mean=0.327 active=5.6/12


Epoch 35 (Stage 2):  93%|█████████▎| 201/216 [02:15<00:10,  1.50it/s, loss=0.726, G=0.685, L=0.016, C=0.001, D=0.017, cf=0.000, lr=1.4e-05]


  [Conf] min=0.007 max=0.993 mean=0.324 active=5.4/12


Validation: 100%|██████████| 33/33 [03:39<00:00,  6.65s/it]



Epoch 35 Summary:
  Train Loss: 0.7807
    Global: 0.7388
    Local: 0.0155
    Consistency: 0.0012
    Diversity: 0.0174
    Conf Floor: 0.0019
  Val Loss: 1.5772
Final model saved: outputs\gfa_111k\clip4cad_gfa_final.pt

Training Complete!
✓ Training completed!


## Cell 7: Resume Training from Checkpoint (Optional)

Resume training from a specific checkpoint epoch.

In [ ]:
# Specify the checkpoint to resume from
checkpoint_epoch = 10  # Change this to the epoch you want to resume from
checkpoint_path = output_dir / f"checkpoint_epoch{checkpoint_epoch}.pt"

if checkpoint_path.exists():
    print(f"Resuming from {checkpoint_path}...")
    trainer.load_checkpoint(str(checkpoint_path))
    print(f"✓ Checkpoint loaded from epoch {checkpoint_epoch}")
    
    print("\nRestarting training...")
    print("="*60)
    trainer.train()
    print("="*60)
    print("✓ Training completed!")
else:
    print(f"❌ Checkpoint not found: {checkpoint_path}")
    print(f"\nAvailable checkpoints:")
    checkpoints = sorted(output_dir.glob("checkpoint_epoch*.pt"))
    if checkpoints:
        for ckpt in checkpoints:
            print(f"  - {ckpt.name}")
    else:
        print("  - No checkpoints found")

## Cell 8: Quick Validation (Optional)

Run validation without training to check model performance.

In [ ]:
print("Running validation...")
val_metrics = trainer.validate()

print("\n✓ Validation Results:")
print("="*60)
for key, value in val_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")
print("="*60)

## Cell 9: Check RAM Usage (Optional)

Monitor memory usage to ensure no memory leaks.

In [ ]:
import psutil
import gc

# Force garbage collection
gc.collect()
torch.cuda.empty_cache()

# Check system RAM
process = psutil.Process()
ram_gb = process.memory_info().rss / 1e9
total_ram = psutil.virtual_memory().total / 1e9
available_ram = psutil.virtual_memory().available / 1e9

print("Memory Usage:")
print("="*60)
print(f"  Process RAM: {ram_gb:.1f} GB")
print(f"  Total System RAM: {total_ram:.1f} GB")
print(f"  Available RAM: {available_ram:.1f} GB")
print(f"  RAM Usage: {(total_ram - available_ram) / total_ram * 100:.1f}%")

# Check GPU memory if available
if torch.cuda.is_available():
    gpu_allocated = torch.cuda.memory_allocated() / 1e9
    gpu_reserved = torch.cuda.memory_reserved() / 1e9
    gpu_total = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print(f"\nGPU Memory:")
    print(f"  Allocated: {gpu_allocated:.1f} GB")
    print(f"  Reserved: {gpu_reserved:.1f} GB")
    print(f"  Total: {gpu_total:.1f} GB")
    print(f"  Usage: {gpu_allocated / gpu_total * 100:.1f}%")

print("="*60)

In [36]:
# Reload model module to pick up code changes (no data reload needed!)
import importlib
from clip4cad.models import clip4cad_gfa

# Reload the module with the fix
importlib.reload(clip4cad_gfa)

# Re-import the class
from clip4cad.models.clip4cad_gfa import CLIP4CAD_GFA
from clip4cad.training import gfa_trainer                                                                                                                                                                                                  
importlib.reload(gfa_trainer)
from clip4cad.training.gfa_trainer import GFATrainer

from clip4cad.training import hard_negative_mining                                                                                                                                                                                         
importlib.reload(hard_negative_mining)
print("✓ Model code reloaded with confidence clamp fix")

✓ Model code reloaded with confidence clamp fix
